In [5]:
import plotly.graph_objects as go
import numpy as np
import os

# モデルのパスや埋め込み範囲などのパラメータの設定
model_path = '/home/lab/eight/KGE-HAKE/models/HAKE_wn18rr_0'
embedding_range = 0.01
head_id = 26440 #3408	united_states.n.01
tail_id = 9855	#26440 new_york.n.01 9855	new_york.n.02
num_bins = 100  # ヒストグラムのビンの数

# エンティティ名の辞書読み込み
entities_dict_file = '/home/lab/eight/KGE-HAKE/data/wn18rr_text/entities.dict'
entities_dict = {}
with open(entities_dict_file, 'r', encoding='utf-8') as f:
    for line in f:
        parts = line.strip().split('\t')
        entities_dict[int(parts[0])] = parts[1]

In [6]:
# エンティティの埋め込みをロード
entity_embedding = np.load(os.path.join(model_path, 'entity_embedding.npy'))

# HeadとTailの座標計算
head = entity_embedding[head_id]
tail = entity_embedding[tail_id]

phase_head, mod_head = np.split(head, 2)
phase_tail, mod_tail = np.split(tail, 2)

mod_head = np.log(np.abs(mod_head)) * np.sign(mod_head)
mod_tail = np.log(np.abs(mod_tail)) * np.sign(mod_tail)

phase_head = phase_head / embedding_range * np.pi
phase_tail = phase_tail / embedding_range * np.pi

x_head, y_head = mod_head * np.cos(phase_head), mod_head * np.sin(phase_head)
x_tail, y_tail = mod_tail * np.cos(phase_tail), mod_tail * np.sin(phase_tail)

fig = go.Figure()

# Head Entityのプロット
fig.add_trace(go.Scatter(x=x_head, y=y_head, mode='markers', name="head "+entities_dict[head_id], text=[f'Head ID: {head_id}'] * len(x_head)))
# Tail Entityのプロット
fig.add_trace(go.Scatter(x=x_tail, y=y_tail, mode='markers', name="tail "+entities_dict[tail_id], text=[f'Tail ID: {tail_id}'] * len(x_tail)))

# レイアウトの設定
fig.update_layout(
    title="Visualizationof theembeddingsofseveralentity pairs",
    xaxis_title=" ",
    yaxis_title=" ",
    showlegend=True,
    width=500,  # 幅
    height=500,  # 高さ
    xaxis=dict(scaleanchor="y", scaleratio=1),  # x軸のアスペクト比を1:1に設定
    yaxis=dict(scaleanchor="x", scaleratio=1),  # y軸のアスペクト比を1:1に設定
)

# プロットの表示
fig.show()


In [7]:
# エンティティの埋め込みをロード
entity_embedding = np.load(os.path.join(model_path, 'entity_embedding.npy'))

# headとtailの各座標を計算
phase_head, mod_head = np.split(entity_embedding[head_id], 2)
phase_tail, mod_tail = np.split(entity_embedding[tail_id], 2)

# ログ変換と極座標への変換
mod_head = np.log(np.abs(mod_head)) * np.sign(mod_head)
mod_tail = np.log(np.abs(mod_tail)) * np.sign(mod_tail)

phase_head = phase_head / embedding_range * np.pi
phase_tail = phase_tail / embedding_range * np.pi

# 極座標から直交座標への変換
x_head, y_head = mod_head * np.cos(phase_head), mod_head * np.sin(phase_head)
x_tail, y_tail = mod_tail * np.cos(phase_tail), mod_tail * np.sin(phase_tail)

# 距離の計算
distance_head = np.sqrt(x_head**2 + y_head**2)
distance_tail = np.sqrt(x_tail**2 + y_tail**2)

# ヒストグラムの作成
hist_head = np.histogram(distance_head, bins=num_bins, range=(0, np.max(distance_head)))
hist_tail = np.histogram(distance_tail, bins=num_bins, range=(0, np.max(distance_tail)))

# Plotlyを使用してヒストグラムをプロット
fig = go.Figure()
fig.add_trace(go.Bar(x=hist_head[1][:-1], y=hist_head[0], name="head "+entities_dict[head_id]))

# Tail Entityのヒストグラム
fig.add_trace(go.Bar(x=hist_tail[1][:-1], y=hist_tail[0], name="tail "+entities_dict[tail_id]))


# レイアウトの設定
fig.update_layout(
    title='Histograms of the modulus of entity embeddings',
    xaxis=dict(title='原点からの距離'),
    yaxis=dict(title='カウント'),
    barmode='overlay',  # HeadとTailのヒストグラムをオーバーレイ
)

# プロットの表示
fig.show()


In [8]:
# 平均距離の計算
mean_distance_head = np.mean(np.sqrt(x_head**2 + y_head**2))
mean_distance_tail = np.mean(np.sqrt(x_tail**2 + y_tail**2))

# 結果の出力
print(f"平均距離（Head）: {mean_distance_head}")
print(f"平均距離（Tail）: {mean_distance_tail}")

平均距離（Head）: 3.90541934967041
平均距離（Tail）: 4.2313923835754395
